# VHR Canopy Height Modeling Workflow Demo
This is a demo of the `VHR Toolkit` for preparing $SR_{VHR}$ for the $CHM_{VHR}$ Workflow  

Paul Montesano  
Jordan Caraballo-Vega  
Matthew Macander  
Mark Carroll    
Glenn Tamkin  
Caleb Spradlin  

Repositories:  
https://github.com/nasa-nccs-hpda/above-shrubs.git


`Motivations:`

 - Demo the pre-processing of VHR used to run through the $CHM_{VHR}$ workflow ; use for a list of catids

`Notes:`  
 - need to ssh into ilab or forest VMs


## Prep

Notes doc (TODO)  
https://docs.google.com/document/d/17gaIal1czeee1b09FixWu0GKAWVvR2yodU9fmwKuGXE/edit

In [1]:
MAINDIR = '/explore/nobackup/projects/above/misc/ABoVE_Shrubs'

## Choose a catalog id: $catalogID_{MAXAR}$

In [2]:
CATID = '1010010000C41400'

## Query NASA NGA database of MAXAR $M1BS_{VHR}$ imagery
this creates a list the input $M1BS_{VHR}$ scenes found for a $catalogID_{MAXAR}$ in our NASA NGA dataBase archive into a local dir of choice in preparation for workflow processing
#### `query_db_catid` works with `forest` conda env which has `psycopg2`
Resources:  
 - code/geoscitools/query_ngadb.ipynb
 - Catalog IDs of interest:
   - Western AK (July,August): https://docs.google.com/spreadsheets/d/1XdSTInm18KYBUvaftyev449GwhMKZzWjl3uV1gUF-yI/edit#gid=288708282
   - Western AK (July,August), Sun Elev > 17: https://docs.google.com/spreadsheets/d/1ig4YQu1dG-2pWWbl6ZRxz37ebERsX80bLcEZLIdKqtQ/edit#gid=1404456155

In [3]:
import sys
sys.path.append('/home/pmontesa/code/evhr')
import os
import query_db_catid
import pandas as pd
import glob

In [4]:
# TODO - get this right. 
# The expected ENV variables arent accessible for some reason
#
#  NGA DB password doesnt appear when printed here, but it does in terminal
#!env | grep NGADBPASS
#!source /home/pmontesa/.bashrc
#os.environ['NGADBPASS']

In [21]:
NGADBPASS = '----' # env | grep NGADBPASS  type this at terminal and copy result

In [6]:
OUTDIR_QUERY = os.path.join(MAINDIR, 'query_nga_db')
!mkdir -p $OUTDIR_QUERY

In [7]:
# Wrappers of query
def GET_NGA_CATID_SCENE_SYMLINKS(CATID, OUTDIR_QUERY):
    OUTDIR_CATID = os.path.join(OUTDIR_QUERY, CATID)
    !mkdir -p $OUTDIR_CATID
    query_db_catid.query_db_catid_v2(CATID, 'M1BS', OUTDIR_CATID, None,'pmontesa', NGADBPASS, symlink=True)
    return OUTDIR_CATID

def GET_NGA_CATID_SCENE_LIST(CATID, OUTDIR_QUERY):
    OUT_SCENE_LIST_FN = os.path.join(OUTDIR_QUERY, CATID + '_scenes.csv')
    
    query_db_catid.query_db_catid_v2(CATID, 'M1BS', OUTDIR_QUERY, OUT_SCENE_LIST_FN,'pmontesa', NGADBPASS, symlink=False)
    return OUT_SCENE_LIST_FN

In [10]:
# Run a single query to return data w
#GET_NGA_CATID_SCENE_SYMLINKS(CATID, OUTDIR_QUERY)

In [13]:
SCENES_FILE = GET_NGA_CATID_SCENE_LIST(CATID, OUTDIR_QUERY)
SCENES_FILE


	 Now executing database query on catID '1010010000C41400' ...

	 Found '2' scenes for catID '1010010000C41400' 

	 List of all prod_id '['505853386050']'
	 Selecting data associated with prod_id '505853386050'
/css/nga/QB02/1B/2002/191/QB02_1010010000C41400_M1BS_505853386050_01/QB02_20020710215038_1010010000C41400_02JUL10215038-M1BS-505853386050_01_P001.ntf
/css/nga/QB02/1B/2002/191/QB02_1010010000C41400_M1BS_505853386050_01/QB02_20020710215041_1010010000C41400_02JUL10215041-M1BS-505853386050_01_P002.ntf


'/explore/nobackup/projects/above/misc/ABoVE_Shrubs/query_nga_db/1010010000C41400_scenes.csv'

## VHR TOA Processing (with eVHR tool)
this produces $TOA_{VHR}$, an orthorectified top-of-atmosphere reflectance image of the multi-spectral bands of the original $M1BS_{VHR}$ input

In [16]:
OUTDIR_TOA = os.path.join(MAINDIR, 'toa','002m')

#### $TOA_{VHR}$: confirm output exists after run

In [17]:
SEARCH_TOA = os.path.join(OUTDIR_TOA, f'*{CATID}*.tif')
SEARCH_TOA

'/explore/nobackup/projects/above/misc/ABoVE_Shrubs/toa/002m/*1010010000C41400*.tif'

In [18]:
glob.glob(SEARCH_TOA)

[]

In [19]:
!echo \
singularity exec -B /explore,/panfs,/css,/nfs4m \
/explore/nobackup/people/iluser/ilab_containers/evhr_4.0.0.sif \
python /usr/local/ilab/evhr/view/evhrToaCLV.py \
-o $OUTDIR_TOA \
--scenes_in_file $SCENES_FILE

singularity exec -B /explore,/panfs,/css,/nfs4m /explore/nobackup/people/iluser/ilab_containers/evhr_4.0.0.sif python /usr/local/ilab/evhr/view/evhrToaCLV.py -o /explore/nobackup/projects/above/misc/ABoVE_Shrubs/toa/002m --scenes_in_file /explore/nobackup/projects/above/misc/ABoVE_Shrubs/query_nga_db/1010010000C41400_scenes.csv


## VHR Cloudmask Processing
this produces a $Cloudmask_{VHR}$ from the input $TOA_{VHR}$  
https://nasa-nccs-hpda.github.io/vhr-cloudmask/latest/readme.html#development-pipeline-details

In [107]:
OUTDIR_CLOUD = os.path.join(MAINDIR, 'cloudmask','002m')

### run: compile the call to invoke it at terminal
login to `ilab201-210` or `forest201-210`

In [108]:
!echo \
singularity exec --nv -B $NOBACKUP,/explore/nobackup/people,/explore/nobackup/projects \
/explore/nobackup/projects/ilab/containers/vhr-cloudmask.sif vhr-cloudmask-cli \
-o $OUTDIR_CLOUD \
-r $SEARCH_CLOUDMASK \
-s predict

singularity exec --nv -B ,/explore/nobackup/people,/explore/nobackup/projects /explore/nobackup/projects/ilab/containers/vhr-cloudmask.sif vhr-cloudmask-cli -o -r -s predict


In [109]:
SEARCH_CLOUDMASK = os.path.join(OUTDIR_CLOUD, f'*{CATID}*.tif')
SEARCH_CLOUDMASK

'/explore/nobackup/projects/above/misc/ABoVE_Shrubs/cloudmask/002m/*1010010000E2CF00*.tif'

#### $Cloudmask_{VHR}$: confirm output exists after run

In [110]:
glob.glob(SEARCH_CLOUDMASK)

[]

## VHR SRLite Processing
this produces a $SR_{VHR}$ that is modeled from input $TOA_{VHR}$ and $SR_{reference}$

In [111]:
OUTDIR_SR = os.path.join(MAINDIR, 'srlite','002m')
OUTPUT_SR_CATID_INFO_FN = os.path.join(OUTDIR_SR, f'{CATID}.out')
OUTPUT_SR_CATID_INFO_FN

'/explore/nobackup/projects/above/misc/ABoVE_Shrubs/srlite/002m/1010010000E2CF00.out'

In [112]:
OUTDIR_CCDC = os.path.join(MAINDIR, 'ccdc','030m')
SEARCH_CCDC = os.path.join(OUTDIR_CCDC, f'*{CATID}*.tif')
SEARCH_CCDC

'/explore/nobackup/projects/above/misc/ABoVE_Shrubs/ccdc/030m/*1010010000E2CF00*.tif'

#### $SR_{reference}$: confirm the reference SR data exists (using CCDC as reference)
this $SR_{reference}$ data along with $TOA_{VHR}$ is required to model $SR_{VHR}$

In [113]:
glob.glob(SEARCH_CCDC)

[]

In [93]:
!echo \
singularity run -B /explore,/panfs \
/explore/nobackup/people/iluser/ilab_containers/srlite_1.1.0.sif \
python /usr/local/ilab/srlite/view/SrliteWorkflowCommandLineView.py \
-toa_dir $OUTDIR_TOA \
-target_dir $OUTDIR_CCDC \
-cloudmask_dir $OUTDIR_CLOUD \
-bandpairs "[['blue_ccdc', 'BAND-B'], ['green_ccdc', 'BAND-G'], ['red_ccdc', 'BAND-R'], ['nir_ccdc', 'BAND-N'],['blue_ccdc', 'BAND-C'], ['green_ccdc', 'BAND-Y'], ['red_ccdc', 'BAND-RE'], ['nir_ccdc', 'BAND-N2']]" \
-output_dir $OUTDIR_SR \
--regressor rma --debug 1 --pmask --cloudmask --csv --band8 \
--cloudmask_suffix toa.cloudmask.tif --target_suffix ccdc.tif \
2>&1 | tee -a $OUTPUT_SR_CATID_INFO_FN


tee: /explore/nobackup/projects/above/misc/ABoVE_Shrubs/srlite/002m/10300100045FA300.out: Permission denied
singularity run -B /explore,/panfs /explore/nobackup/people/iluser/ilab_containers/srlite_1.1.0.sif python /usr/local/ilab/srlite/view/SrliteWorkflowCommandLineView.py -toa_dir /explore/nobackup/projects/above/misc/ABoVE_Shrubs/toa/002m -target_dir /explore/nobackup/projects/above/misc/ABoVE_Shrubs/ccdc/030m -cloudmask_dir /explore/nobackup/projects/above/misc/ABoVE_Shrubs/cloudmask/002m -bandpairs [['blue_ccdc', 'BAND-B'], ['green_ccdc', 'BAND-G'], ['red_ccdc', 'BAND-R'], ['nir_ccdc', 'BAND-N'],['blue_ccdc', 'BAND-C'], ['green_ccdc', 'BAND-Y'], ['red_ccdc', 'BAND-RE'], ['nir_ccdc', 'BAND-N2']] -output_dir /explore/nobackup/projects/above/misc/ABoVE_Shrubs/srlite/002m --regressor rma --debug 1 --pmask --cloudmask --csv --band8 --cloudmask_suffix toa.cloudmask.tif --target_suffix ccdc.tif
